In [1]:
!/usr/local/Cellar/jupyterlab/3.0.0_1/libexec/bin/python -m pip install tabula-py pandas

In [303]:
import tabula
import pandas as pd
import io
import math

In [304]:
pd.options.display.max_colwidth = 1000
pd.options.display.max_rows = None

In [305]:
filename = "/Users/breno/Desktop/Padrao_TISS_Componente_Organizacional__202012.pdf"
first_page = 79
last_page = 84 + 1  # need to add 1 since end index is exclusive
pages = list(range(first_page, last_page))
dfs = tabula.read_pdf(filename, pages=pages)[1:]

In [306]:
dfs

[  Unnamed: 0                 Tabela de Categoria do Padrão TISS
 0     Código                             Descrição da categoria
 1          1                          Componente Organizacional
 2          2                 Componente de Conteúdo e Estrutura
 3          3  Componente de Representação de Conceitos em Saúde,
      4  \
 0    5   
 1   18   
 2   19   
 3   20   
 4   22   
 5   23   
 6   24   
 7   25   
 8   26   
 9   27   
 10  28   
 11  29   
 12  30   
 13  31   
 14  32   
 15  33   
 16  34   
 17  35   
 18  36   
 19  37   
 20  38   
 21  39   
 22  40   
 23  41   
 24  42   
 25  43   
 
                                                    Componente de Comunicação  
 0                                      Componente de Segurança e Privacidade  
 1                          Terminologia de diárias, taxas e gases medicinais  
 2                                           Terminologia de materiais e OPME  
 3                                               Termin

In [307]:
# Fix the first dataframe (which contains the header)
df = dfs[0]
headers = df.iloc[0]
dfs[0]  = pd.DataFrame(df.values[1:], columns=headers)
dfs[0]

,Código,Descrição da categoria
0,1,Componente Organizacional
1,2,Componente de Conteúdo e Estrutura
2,3,Componente de Representação de Conceitos em Saúde


In [308]:
# Set the headers in the remaining dataframes

def set_headers(df, headers):
    new_df = pd.read_csv(io.StringIO(df.to_csv(index=False)), header=None)
    return pd.DataFrame(new_df.values, columns=headers)

for i in range(1, len(dfs)):
    dfs[i] = process_df(dfs[i], headers)

In [309]:
df = pd.concat(dfs, ignore_index=True)

In [310]:
df.columns

Index(['Código', 'Descrição da categoria'], dtype='object', name=0)

In [311]:
df

,Código,Descrição da categoria
0,1,Componente Organizacional
1,2,Componente de Conteúdo e Estrutura
2,3,Componente de Representação de Conceitos em Saúde
3,4,Componente de Comunicação
4,5,Componente de Segurança e Privacidade
5,18,"Terminologia de diárias, taxas e gases medicinais"
6,19,Terminologia de materiais e OPME
7,20,Terminologia de medicamentos
8,22,Terminologia de procedimentos e eventos em saúde
9,23,Terminologia de caráter do atendimento


In [312]:
new_df = pd.DataFrame(columns=df.columns)

i = 65
suffix = df.iloc[i]['Código']
row = df.iloc[i-1].copy()
row['Descrição da categoria'] += ' {}'.format(suffix)
new_df = new_df.append(row)
new_df = new_df.reset_index(drop=True)
new_df

,Código,Descrição da categoria
0,102,Legenda da mensagem de solicitação de autorização para realização de procedimentos


In [323]:
df.shape

(138, 2)

In [329]:
'''
Tabula does not deal well with entries that take up more
than one line in the PDF.
We need to scan the df in order to find these entries
and fix them up.
'''

'''
def fix_entries(df):
    i = 0
    while i < df.shape[0]:
        x = df.iloc[i]['Descrição da categoria']
        if isinstance(x, float) and math.isnan(x):
            print()
            print(df.iloc[i-1])
            print(df.iloc[i])
            suffix = df.iloc[i]['Código']
            df.iloc[i-1]['Descrição da categoria'] = df.iloc[i-1]['Descrição da categoria'] + ' {}'.format(suffix)
            print(df.iloc[i-1])
            print('Dropping {}'.format(i))
            df.drop(i, inplace=True)
            df = df.reset_index(drop=True)
        else:
            i += 1
'''

def fix_entries(df):
    new_df = pd.DataFrame(columns=df.columns)
    i = 1
    while i < df.shape[0]:
        row = df.iloc[i-1].copy()
        x = df.iloc[i]['Descrição da categoria']
        if isinstance(x, float) and math.isnan(x):
            suffix = df.iloc[i]['Código']
            row['Descrição da categoria'] += ' {}'.format(suffix)
            i += 1
        new_df = new_df.append(row, ignore_index=True)
        i += 1
        
    row = df.iloc[df.shape[0]-1].copy()  # copy last row
    new_df.append(row, ignore_index=True)
    return new_df

new_df = fix_entries(df)

In [332]:
# We can now export to CSV
new_df.to_csv('2-output.csv', index=False)